In [1]:
# importing libraries
# %pip install mediapipe opencv-python matplotlib
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Capturing the expected poses in a numpy array

In [2]:
# setting workout timer to 30 seconds
timeout = 5
# initializing the Landmarks array
correct_landmarks = np.array([])

In [3]:
workout_time = time.time() + timeout + 10   # 30 seconds from now (10 seconds for the user to get ready)
cap = cv2.VideoCapture('workout.avi')
#recording video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
record = cv2.VideoWriter('expected.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
# setting up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened() and time.time() < workout_time:
        ret, frame = cap.read()
        if not ret:
            print("Video Ended")
            break
        # detect pose landmarks and render them on the image
        # convert the image from BGR to RGB (opneCV uses BGR by default)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # make detection from pose instance
        results = pose.process(image)
        correct_landmarks = np.append(correct_landmarks, results.pose_landmarks)
        # print(results.pose_landmarks)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # print(results)
        # render pose landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(51,255,51), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(51,153,255), thickness=2, circle_radius=2))
        
        # cv2.imshow('MediaPipe Pose', frame)
        cv2.imshow('MediaPipe Pose', image)
        # recording video
        record.write(image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    record.release()
    cv2.destroyAllWindows() 
    print("Correct pose recorded successfully!")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThrea

Correct pose recorded successfully!


In [4]:
# correct_landmarks
np.save('correct_landmarks.npy', correct_landmarks)
# loading the correct landmarks
saved_landmarks = np.load('correct_landmarks.npy', allow_pickle=True)
print(f"shape of correct_landmarks: {correct_landmarks.shape}")
#printing few landmarks:
for i in range(3):
    print(f"Pose {i}")
    for j in range(5):
        print(correct_landmarks[i].landmark[j])

shape of correct_landmarks: (152,)
Pose 0
x: 0.5071608424186707
y: 0.13241057097911835
z: -0.6103724837303162
visibility: 0.9999938011169434

x: 0.5147470831871033
y: 0.1213073581457138
z: -0.5967898964881897
visibility: 0.9999772310256958

x: 0.5192680358886719
y: 0.12022244930267334
z: -0.5968376398086548
visibility: 0.9999786615371704

x: 0.5234243273735046
y: 0.11925651878118515
z: -0.5969747304916382
visibility: 0.9999788999557495

x: 0.5009242296218872
y: 0.12191130220890045
z: -0.5938164591789246
visibility: 0.9999672174453735

Pose 1
x: 0.5123035907745361
y: 0.13222724199295044
z: -0.4960736930370331
visibility: 0.9999942779541016

x: 0.5197516679763794
y: 0.12128492444753647
z: -0.4830795228481293
visibility: 0.9999791383743286

x: 0.5240457057952881
y: 0.12022306770086288
z: -0.48309507966041565
visibility: 0.9999804496765137

x: 0.5280026197433472
y: 0.11927000433206558
z: -0.4832000732421875
visibility: 0.9999805688858032

x: 0.5068050622940063
y: 0.12173807621002197
z: -0.

In [ ]:
# workout_time = time.time() + timeout 
# correct_landmarks = pd.read_csv('correct_landmarks.csv').to_numpy()  
count = 0
poses_matched = 0
cap = cv2.VideoCapture(0)
#recording video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
record = cv2.VideoWriter('actual.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Workout Ended!")
            break
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # make detection from pose instance
        current_results = pose.process(image)
        # print(results.pose_landmarks)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # getting the correct pose landmarks
        correct_pose_landmark = correct_landmarks[count % saved_landmarks.shape[0]]
        count += 1
        #compare the results
        match_count = 0
        for (act, expected) in zip(current_results.pose_landmarks.landmark, correct_pose_landmark.landmark):
            # calculate the distance between the actual and expected landmarks
            dist = np.sqrt((act.x - expected.x)**2 + (act.y - expected.y)**2 + (act.z - expected.z)**2)
            if dist < 0.25:
                match_count += 1
        # calculate the percentage of matched landmarks
        match_percent = (match_count / len(correct_pose_landmark.landmark)) * 100
        text = f"{round(match_percent, 2)}% match - "  
        # render pose landmarks on the image
        if match_percent > 75:
            text += "Correct Pose"
            color = (0, 255, 0)
            poses_matched += 1
        else:
            text+= "Incorrect Pose"
            color = (0, 0, 255)
            mp_drawing.draw_landmarks(image, correct_pose_landmark, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(51,255,51), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(51,153,255), thickness=2, circle_radius=2))    
            
        mp_drawing.draw_landmarks(image, current_results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
        cv2.putText(image,text, (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('MediaPipe Pose', image)
        record.write(image)
        
        # render the results from previous cell
        sentence = "No pose detected"
        if cv2.waitKey(10) & 0xFF == ord('q') :
            break
    cap.release()
    record.release()
    cv2.destroyAllWindows() 

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread (0x37de710).
Cannot move to target thread (0x2d96e20)

QObject::moveToThread: Current thread (0x2d96e20) is not the object's thread

In [10]:
# print the results
print(f"Total poses matched: {poses_matched}")
print(f"Total poses attempted: {count}")
print(f"Accuracy: {round(poses_matched/count * 100, 2)}%")

Total poses matched: 0
Total poses attempted: 229
Accuracy: 0.0%
